<a href="https://colab.research.google.com/github/saisravyaa/survival-prediction/blob/main/survivalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
dt=pd.read_csv("/content/proj.csv")

In [ ]:
dt.head()

,id,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,...,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
0,1,1,1,22.830137,0,1,0,1,1,1,...,1,7.20,1.33876,5.38,35,19,51,32,999,0
1,2,1,0,23.342466,0,1,0,-1,-1,1,...,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
2,3,1,0,26.394521,0,1,0,-1,-1,1,...,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
3,4,0,0,39.684932,1,1,0,1,2,1,...,?,4.25,29.481647,0.14,50,23,29,19,53,1
4,5,0,1,33.358904,0,0,0,1,2,0,...,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0


In [ ]:
dt=dt.drop(['id'], axis=1)


In [ ]:
dt.shape

(187, 37)

In [ ]:
dt.dtypes

Recipientgender           int64
Stemcellsource            int64
Donorage                float64
Donorage35                int64
IIIV                      int64
Gendermatch               int64
DonorABO                  int64
RecipientABO             object
RecipientRh              object
ABOmatch                 object
CMVstatus                object
DonorCMV                 object
RecipientCMV             object
Disease                  object
Riskgroup                 int64
Txpostrelapse             int64
Diseasegroup              int64
HLAmatch                  int64
HLAmismatch               int64
Antigen                  object
Alel                     object
HLAgrI                    int64
Recipientage            float64
Recipientage10            int64
Recipientageint           int64
Relapse                   int64
aGvHDIIIIV                int64
extcGvHD                 object
CD34kgx10d6             float64
CD3dCD34                 object
CD3dkgx10d8              object
Rbodymas

In [ ]:
dt.dropna()

,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,ABOmatch,...,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
0,1,1,22.830137,0,1,0,1,1,1,0,...,1,7.20,1.33876,5.38,35,19,51,32,999,0
1,1,0,23.342466,0,1,0,-1,-1,1,0,...,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
2,1,0,26.394521,0,1,0,-1,-1,1,0,...,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
3,0,0,39.684932,1,1,0,1,2,1,1,...,?,4.25,29.481647,0.14,50,23,29,19,53,1
4,0,1,33.358904,0,0,0,1,2,0,1,...,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,1,1,37.575342,1,1,0,1,1,0,0,...,1,11.08,2.52275,4.39,44,15,22,16,385,1
183,0,1,22.895890,0,0,0,1,0,1,1,...,1,4.64,1.038858,4.47,44.5,12,30,1000000,634,1
184,0,1,27.347945,0,1,0,1,-1,1,1,...,1,7.73,1.635559,4.73,33,16,16,1000000,1895,0
185,1,1,27.780822,0,1,0,1,0,1,1,...,0,15.41,8.07777,1.91,24,13,14,54,382,1


In [ ]:
cor=abs(dt.corr(method="pearson"))
relevant_features = cor[cor<0.5]
list4=relevant_features.keys().to_list()
list4

<ipython-input-10-f190f6e93efb>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor=abs(dt.corr(method="pearson"))


['Recipientgender',
 'Stemcellsource',
 'Donorage',
 'Donorage35',
 'IIIV',
 'Gendermatch',
 'DonorABO',
 'Riskgroup',
 'Txpostrelapse',
 'Diseasegroup',
 'HLAmatch',
 'HLAmismatch',
 'HLAgrI',
 'Recipientage',
 'Recipientage10',
 'Recipientageint',
 'Relapse',
 'aGvHDIIIIV',
 'CD34kgx10d6',
 'ANCrecovery',
 'PLTrecovery',
 'time_to_aGvHD_III_IV',
 'survival_time',
 'survival_status']

In [ ]:
coloums=['Recipientgender','Stemcellsource','Donorage','Donorage35','IIIV','Gendermatch','DonorABO','Riskgroup','Txpostrelapse','Diseasegroup','HLAmatch','HLAmismatch','HLAgrI','Recipientage','Recipientage10','Recipientageint','Relapse','aGvHDIIIIV','CD34kgx10d6','ANCrecovery','PLTrecovery','time_to_aGvHD_III_IV','survival_time',]
x=dt[coloums]
y=dt["survival_status"]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [ ]:
x

array([[ 0.81831709,  0.53819622, -1.28998124, ..., -0.31616067,
        -1.8580561 ,  0.07111487],
       [ 0.81831709, -1.85805838, -1.22787834, ..., -0.31620937,
         0.53819622, -0.91553139],
       [ 0.81831709, -1.85805838, -0.85791788, ..., -0.3162685 ,
         0.53819622, -0.5945173 ],
       ...,
       [-1.22202019,  0.53819622, -0.74234683, ..., -0.31628242,
         0.53819622,  1.12857307],
       [ 0.81831709,  0.53819622, -0.68987485, ..., -0.31628938,
        -1.85800338, -0.65706784],
       [ 0.81831709,  0.53819622,  2.67663232, ..., -0.3162685 ,
         0.53819622,  0.20093675]])

In [ ]:
df = pd.DataFrame(x)
df.to_csv("processed_data.csv")

In [ ]:
y.value_counts()

0    102
1     85
Name: survival_status, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
x,y=sm.fit_resample(x, y)
y.value_counts()

0    102
1    102
Name: survival_status, dtype: int64

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state=1,train_size=0.8)

In [ ]:
rnn = Sequential()

In [ ]:
rnn.add(LSTM(units = 45, return_sequences = True, input_shape = (train_X.shape[1], 1)))

In [ ]:
rnn.add(Dropout(0.2))

In [ ]:
rnn.add(LSTM(units = 45, return_sequences = True))

rnn.add(Dropout(0.2))

rnn.add(LSTM(units = 45, return_sequences = True))

rnn.add(Dropout(0.2))

rnn.add(LSTM(units = 45))

rnn.add(Dropout(0.2))

In [ ]:
rnn.add(Dense(units = 1))

In [ ]:
rnn.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
rnn.fit(train_X,train_y, epochs = 100, batch_size = 32)

Epoch 1/100
6/6 [==============================] - 10s 55ms/step - loss: 0.4401
Epoch 2/100
6/6 [==============================] - 0s 54ms/step - loss: 0.2949
Epoch 3/100
6/6 [==============================] - 0s 57ms/step - loss: 0.2439
Epoch 4/100
6/6 [==============================] - 0s 56ms/step - loss: 0.2672
Epoch 5/100
6/6 [==============================] - 0s 54ms/step - loss: 0.2679
Epoch 6/100
6/6 [==============================] - 0s 55ms/step - loss: 0.2514
Epoch 7/100
6/6 [==============================] - 0s 56ms/step - loss: 0.2471
Epoch 8/100
6/6 [==============================] - 0s 55ms/step - loss: 0.2470
Epoch 9/100
6/6 [==============================] - 0s 56ms/step - loss: 0.2452
Epoch 10/100
6/6 [==============================] - 0s 53ms/step - loss: 0.2464
Epoch 11/100
6/6 [==============================] - 0s 56ms/step - loss: 0.2439
Epoch 12/100
6/6 [==============================] - 0s 55ms/step - loss: 0.2533
Epoch 13/100
6/6 [==============================

In [ ]:
predictions = rnn.predict(val_X)

2/2 [==============================] - 2s 17ms/step


In [ ]:
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
mse_rnn = MSE(val_y,predictions)

In [ ]:
mse_rnn

0.18306337201282133

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

In [ ]:
print('\nRNN')
print('Accuracy Score:',metrics.accuracy_score(val_y,predictions.round())*100,'%',sep='')
print('Confusion Matrix:',metrics.confusion_matrix(val_y,predictions.round()),sep='\n')
print(classification_report(val_y,predictions.round()))


RNN
Accuracy Score:78.04878048780488%
Confusion Matrix:
[[14  6]
 [ 3 18]]
              precision    recall  f1-score   support

           0       0.82      0.70      0.76        20
           1       0.75      0.86      0.80        21

    accuracy                           0.78        41
   macro avg       0.79      0.78      0.78        41
weighted avg       0.79      0.78      0.78        41



In [ ]:
pip install pyGRNN --ignore-installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from pyGRNN import GRNN

In [ ]:
AGRNN = GRNN(calibration = 'gradient_search')

In [ ]:
AGRNN.fit(train_X,train_y.ravel())

GRNN(bnds=((0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
           (0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
           (0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
           (0, None), (0, None), (0, None), (0, None), (0, None)),
     calibration='gradient_search',
     sigma=array([0.34810787, 0.29480914, 0.59563091, 0.55144653, 0.19241952,
       0.02703171, 0.20325832, 0.78992643, 0.68073025, 0.68093046,
       0.46998804, 0.39658717, 0.42346203, 0.09324004, 0.17276813,
       0.24851571, 0.17095734, 0.20640728, 0.41392028, 0.4       ,
       0.4       , 0.21420016, 0.0209811 ]))

In [ ]:
sigma = AGRNN.sigma

In [ ]:
prediction = AGRNN.predict(val_X)

In [ ]:
mse_AGRNN = MSE(val_y,predictions)
mse_AGRNN

0.18306337201282133

In [ ]:
print('\nGRNN')
print('Accuracy Score:',metrics.accuracy_score(val_y,prediction.round())*100,'%',sep='')
print('Confusion Matrix:',metrics.confusion_matrix(val_y,prediction.round()),sep='\n')
print(classification_report(val_y,prediction.round()))


GRNN
Accuracy Score:90.2439024390244%
Confusion Matrix:
[[17  3]
 [ 1 20]]
              precision    recall  f1-score   support

           0       0.94      0.85      0.89        20
           1       0.87      0.95      0.91        21

    accuracy                           0.90        41
   macro avg       0.91      0.90      0.90        41
weighted avg       0.91      0.90      0.90        41



In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [ ]:
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [ ]:
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
ann.fit(train_X,train_y,batch_size=32,epochs = 75)

Epoch 1/75
6/6 [==============================] - 1s 3ms/step - loss: 0.7349 - accuracy: 0.5092
Epoch 2/75
6/6 [==============================] - 0s 3ms/step - loss: 0.7208 - accuracy: 0.5092
Epoch 3/75
6/6 [==============================] - 0s 3ms/step - loss: 0.7094 - accuracy: 0.5092
Epoch 4/75
6/6 [==============================] - 0s 4ms/step - loss: 0.7013 - accuracy: 0.5153
Epoch 5/75
6/6 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.5399
Epoch 6/75
6/6 [==============================] - 0s 3ms/step - loss: 0.6861 - accuracy: 0.5399
Epoch 7/75
6/6 [==============================] - 0s 3ms/step - loss: 0.6798 - accuracy: 0.5460
Epoch 8/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6752 - accuracy: 0.5460
Epoch 9/75
6/6 [==============================] - 0s 3ms/step - loss: 0.6702 - accuracy: 0.5644
Epoch 10/75
6/6 [==============================] - 0s 3ms/step - loss: 0.6660 - accuracy: 0.5583
Epoch 11/75
6/6 [======================

In [ ]:
predict = ann.predict(val_X)

2/2 [==============================] - 0s 5ms/step


In [ ]:
mse_ann = MSE(val_y,predict)
mse_ann

0.11670248197509701

In [ ]:
print('\nANN')
print('Accuracy Score:',metrics.accuracy_score(val_y,predict.round())*100,'%',sep='')
print('Confusion Matrix:',metrics.confusion_matrix(val_y,predict.round()),sep='\n')
print(classification_report(val_y,predict.round()))


ANN
Accuracy Score:90.2439024390244%
Confusion Matrix:
[[20  0]
 [ 4 17]]
              precision    recall  f1-score   support

           0       0.83      1.00      0.91        20
           1       1.00      0.81      0.89        21

    accuracy                           0.90        41
   macro avg       0.92      0.90      0.90        41
weighted avg       0.92      0.90      0.90        41

